## Import

In [ ]:
import random   # random 모듈을 가져옴 (난수 생성 등에 사용)
import os       # os 모듈을 가져옴 (운영체제와 상호작용에 사용)
import pandas as pd    # pandas 라이브러리를 가져옴 (데이터 분석과 조작에 사용)
import numpy as np     # numpy 라이브러리를 가져옴 (수치 연산에 사용)
from tqdm.auto import tqdm    # tqdm 라이브러리에서 tqdm.auto를 가져옴 (진행 상태를 표시하는데 사용)

from sklearn.preprocessing import LabelEncoder    # scikit-learn 라이브러리에서 LabelEncoder를 가져옴 (라벨 인코딩에 사용)

import torch   # PyTorch 라이브러리를 가져옴 (딥 러닝 프레임워크)
import torch.nn as nn    # PyTorch에서 nn 모듈을 가져옴 (신경망 모델링에 사용)
import torch.optim as optim    # PyTorch에서 optim 모듈을 가져옴 (최적화 알고리즘에 사용)
import torch.nn.functional as F    # PyTorch에서 F 모듈을 가져옴 (신경망 함수에 사용)
from torch.utils.data import Dataset, DataLoader, random_split    # PyTorch에서 Dataset과 DataLoader를 가져옴 (데이터셋과 데이터 로딩에 사용)
from google.colab import drive
drive.mount('/content/drive')
# train data 바로 사용하기 위함(수정)
import warnings
warnings.filterwarnings("ignore")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# device 변수에 GPU 디바이스를 할당하는 코드입니다.
# torch.cuda.is_available() 함수를 사용하여 현재 시스템에서 GPU를 사용할 수 있는지 확인합니다.
# 만약 GPU를 사용할 수 있다면, device에 'cuda' 문자열을 사용하여 GPU 디바이스를 할당합니다.
# 그렇지 않은 경우, 'cpu' 문자열을 사용하여 CPU 디바이스를 할당합니다.
# 이후 모델과 데이터를 device로 올려서 학습 또는 예측을 수행할 때 해당 디바이스를 사용합니다.
# GPU를 사용할 수 있는 경우 GPU를 활용하여 연산을 가속화할 수 있습니다.

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [ ]:
# CFG는 하이퍼파라미터들을 저장하는 딕셔너리입니다.

CFG = {
    'TRAIN_WINDOW_SIZE': 110,  # LSTM 모델에 입력으로 사용될 학습 윈도우의 크기를 나타냅니다. 학습 윈도우의 크기는 90일로 설정되어 있습니다.
    'PREDICT_SIZE': 21,       # LSTM 모델이 예측할 일 수를 나타냅니다. 예측 일 수는 21일로 설정되어 있습니다.
    'EPOCHS': 12,             # 학습 과정에서 전체 데이터셋을 몇 번 반복할지를 나타냅니다. 학습 에포크 수는 10으로 설정되어 있습니다.
    'LEARNING_RATE': 1e-4,    # 학습 과정에서 모델 파라미터를 업데이트하는 데 사용되는 학습률을 나타냅니다. 학습률은 0.0001로 설정되어 있습니다.
    'BATCH_SIZE': 3000,       # 한 번의 학습 스텝에서 사용할 미니배치의 크기를 나타냅니다. 미니배치의 크기는 4096으로 설정되어 있습니다.
    'SEED': 46                # 시드를 설정하기 위해 사용되는 값으로, 랜덤 요소를 고정하여 학습의 재현성을 보장합니다. 시드 값은 41로 설정되어 있습니다.
}


In [ ]:
import random
import os
import numpy as np
import torch

def seed_everything(seed):
    """
    시드 값을 설정하여 랜덤 요소들을 고정하는 함수입니다.

    Args:
        seed (int): 재현성을 위해 사용할 시드 값입니다.
    """
    random.seed(seed)                # random 모듈의 시드 값을 설정합니다.
    os.environ['PYTHONHASHSEED'] = str(seed)  # 파이썬 해시 값의 시드를 설정합니다.
    np.random.seed(seed)             # NumPy 모듈의 시드 값을 설정합니다.
    torch.manual_seed(seed)          # Torch의 CPU 연산을 위한 시드 값을 설정합니다.
    torch.cuda.manual_seed(seed)     # Torch의 GPU 연산을 위한 시드 값을 설정합니다.
    torch.backends.cudnn.deterministic = True  # Torch에서 CuDNN 연산의 재현성을 보장하기 위해 설정합니다.
    torch.backends.cudnn.benchmark = True     # Torch에서 CuDNN 연산을 더 빠르게 수행하도록 설정합니다.

seed_everything(CFG['SEED'])  # CFG 딕셔너리에 저장된 SEED 값으로 시드를 설정합니다.


### 데이터 불러오기

In [ ]:

train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv').drop(columns=['ID', '제품'])

### 데이터 전처리

In [ ]:
# Data Scaling
scale_max_dict = {}
scale_min_dict = {}

for idx in tqdm(range(len(train_data))):
    maxi = np.max(train_data.iloc[idx,4:])
    mini = np.min(train_data.iloc[idx,4:])

    if maxi == mini :
        train_data.iloc[idx,4:] = 0
    else:
        train_data.iloc[idx,4:] = (train_data.iloc[idx,4:] - mini) / (maxi - mini)

    scale_max_dict[idx] = maxi
    scale_min_dict[idx] = mini

  0%|          | 0/15890 [00:00<?, ?it/s]

In [ ]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [ ]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성하는 함수
    data : 일별 판매량 데이터
    train_size : 학습에 활용할 기간의 크기 (기본값은 90)
    predict_size : 추론할 기간의 크기 (기본값은 21)
    '''
    num_rows = len(data)  # 데이터의 총 행 개수를 구합니다.
    window_size = train_size + predict_size  # 학습 기간과 예측 기간을 더한 전체 윈도우 크기를 구합니다.

    # 입력 데이터와 타겟 데이터를 저장할 빈 배열을 생성합니다.
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1))
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])  # 상품 및 지역 인코딩 정보를 배열로 변환합니다.
        sales_data = np.array(data.iloc[i, 4:])  # 판매량 데이터를 배열로 변환합니다.

        for j in range(len(sales_data) - window_size + 1):
            # 현재 윈도우 내의 학습 기간과 판매량 데이터를 잘라서 입력 데이터로 만듭니다.
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]

    return input_data, target_data


In [ ]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성하는 함수
    data : 일별 판매량 데이터
    train_size : 추론을 위해 필요한 일별 판매량 기간 (기본값은 학습에 활용할 기간인 90)
    '''
    num_rows = len(data)  # 데이터의 총 행 개수를 구합니다.

    # 추론을 위한 입력 데이터를 저장할 빈 배열을 생성합니다.
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])  # 상품 및 지역 인코딩 정보를 배열로 변환합니다.
        sales_data = np.array(data.iloc[i, -train_size:])  # 추론을 위해 필요한 일별 판매량 데이터를 배열로 변환합니다.

        # 추론을 위한 윈도우 크기에 해당하는 입력 데이터를 생성합니다.
        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data

    return input_data


In [ ]:
# 학습 데이터(train_data)를 활용하여 학습용 입력 데이터(train_input)와 학습용 타겟 데이터(train_target)를 생성합니다.
train_input, train_target = make_train_data(train_data)

# 학습 데이터(train_data)를 활용하여 추론용 입력 데이터(test_input)를 생성합니다.
test_input = make_predict_data(train_data)


NameError: ignored

In [ ]:
# Train / Validation Split
# train_input과 train_target을 학습용 데이터와 검증용 데이터로 분할합니다.

# 학습용 데이터(train_input, train_target)의 전체 길이를 저장합니다.
data_len = len(train_input)

# 학습용 데이터의 20%를 검증용 데이터로 분할하기 위해, 전체 길이의 20%에 해당하는 부분을 잘라냅니다.
# int(data_len*0.2)는 전체 길이의 20%에 해당하는 정수값을 의미합니다.
# 검증용 데이터의 크기가 전체 길이의 20%가 되도록 학습용 데이터와 검증용 데이터로 나눕니다.
val_input = train_input[-int(data_len*0.2):] # 뒤에서부터 20%에 해당하는 부분을 검증용 데이터로 저장
val_target = train_target[-int(data_len*0.2):] # 뒤에서부터 20%에 해당하는 부분을 검증용 타겟 데이터로 저장

# 검증용 데이터로 잘라낸 나머지 부분은 학습용 데이터로 사용합니다.
train_input = train_input[:-int(data_len*0.2)] # 뒤에서부터 20%를 제외한 부분을 학습용 데이터로 저장
train_target = train_target[:-int(data_len*0.2)] # 뒤에서부터 20%를 제외한 부분을 학습용 타겟 데이터로 저장


In [ ]:
# 학습용 데이터와 검증용 데이터, 그리고 테스트 데이터의 크기(shape)를 확인하는 코드입니다.

# 학습용 데이터(train_input)의 크기를 확인합니다. train_input은 3차원 배열이며, (데이터 개수, 학습 기간, 특성 개수)의 크기를 가집니다.
# 예를 들어, (1000, 90, 5)라면 데이터 개수가 1000개이고, 학습 기간이 90일, 특성 개수가 5개인 학습용 데이터입니다.
# train_input.shape

# 학습용 타겟 데이터(train_target)의 크기를 확인합니다. train_target은 2차원 배열이며, (데이터 개수, 예측 기간)의 크기를 가집니다.
# 예를 들어, (1000, 21)이라면 데이터 개수가 1000개이고, 예측 기간이 21일인 학습용 타겟 데이터입니다.
# train_target.shape

# 검증용 데이터(val_input)의 크기를 확인합니다. val_input은 3차원 배열이며, (데이터 개수, 학습 기간, 특성 개수)의 크기를 가집니다.
# 예를 들어, (200, 90, 5)라면 데이터 개수가 200개이고, 학습 기간이 90일, 특성 개수가 5개인 검증용 데이터입니다.
# val_input.shape

# 검증용 타겟 데이터(val_target)의 크기를 확인합니다. val_target은 2차원 배열이며, (데이터 개수, 예측 기간)의 크기를 가집니다.
# 예를 들어, (200, 21)이라면 데이터 개수가 200개이고, 예측 기간이 21일인 검증용 타겟 데이터입니다.
# val_target.shape

# 테스트 데이터(test_input)의 크기를 확인합니다. test_input은 3차원 배열이며, (데이터 개수, 학습 기간, 특성 개수)의 크기를 가집니다.
# 예를 들어, (500, 90, 5)라면 데이터 개수가 500개이고, 학습 기간이 90일, 특성 개수가 5개인 테스트 데이터입니다.
# test_input.shape

train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

### Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE'], is_inference=False):
        self.data = data.values # convert DataFrame to numpy array
        self.train_size = train_size
        self.predict_size = predict_size
        self.window_size = self.train_size + self.predict_size
        self.is_inference = is_inference

    def __len__(self):
        if self.is_inference:
            return len(self.data)
        else:
            return self.data.shape[0] * (self.data.shape[1] - self.window_size - 3)

    def __getitem__(self, idx):
        if self.is_inference:
            # 추론 시
            encode_info = self.data[idx, :4]
            window = self.data[idx, -self.train_size:]
            input_data = np.column_stack((np.tile(encode_info, (self.train_size, 1)), window))
            return input_data
        else:
            # 학습 시
            row = idx // (self.data.shape[1] - self.window_size - 3)
            col = idx % (self.data.shape[1] - self.window_size - 3)
            encode_info = self.data[row, :4]
            sales_data = self.data[row, 4:]
            window = sales_data[col : col + self.window_size]
            input_data = np.column_stack((np.tile(encode_info, (self.train_size, 1)), window[:self.train_size]))
            target_data = window[self.train_size:]
            return input_data, target_data

### Custom Data Set 생성

In [ ]:
# CustomDataset 인스턴스 생성
dataset = CustomDataset(train_data)

# 전체 데이터셋의 크기
total_size = len(dataset)

# 분리할 데이터셋의 크기 계산
train_size = int(total_size * 0.8)
val_size = total_size - train_size

# random_split 함수를 사용해 데이터셋 분리
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoader 인스턴스 생성
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [ ]:
# # CustomDataset 클래스는 PyTorch의 Dataset 클래스를 상속하여 사용자 정의 데이터셋을 만드는 코드입니다.

# # 클래스 정의 시작
# class CustomDataset(Dataset):
#     # 클래스의 생성자 메서드로, 객체가 생성될 때 호출됩니다.
#     def __init__(self, X, Y):
#         # X와 Y는 학습 데이터와 타겟 데이터를 의미합니다.
#         # X: 학습 데이터, Y: 타겟 데이터
#         self.X = X
#         self.Y = Y

#     # 클래스의 인덱스로 접근할 때 호출되는 메서드입니다.
#     def __getitem__(self, index):
#         # 타겟 데이터가 None이 아닌 경우 (학습 데이터와 타겟 데이터가 모두 제공되는 경우)
#         if self.Y is not None:
#             # 학습 데이터와 타겟 데이터를 각각 Tensor 형태로 변환하여 반환합니다.
#             # index에 해당하는 X와 Y를 Tensor로 변환하여 반환합니다.
#             return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
#         # 타겟 데이터가 None인 경우 (테스트 데이터인 경우)
#         # 학습 데이터만 제공되며, X만 Tensor 형태로 변환하여 반환합니다.
#         return torch.Tensor(self.X[index])

#     # 데이터셋의 총 샘플 개수를 반환하는 메서드입니다.
#     def __len__(self):
#         # 학습 데이터의 총 샘플 개수를 반환합니다.
#         return len(self.X)

# # 클래스 정의 종료


In [ ]:
# CustomDataset을 활용하여 학습 데이터와 검증 데이터에 대한 DataLoader를 생성하는 코드입니다.

# # train_dataset: 학습 데이터셋 객체 생성
# train_dataset = CustomDataset(train_input, train_target)

# # train_loader: 학습 데이터셋을 DataLoader로 묶어줍니다.
# # DataLoader는 배치 크기(batch_size)만큼 데이터를 묶어주는 역할을 합니다.
# # shuffle=True로 설정하여 데이터를 랜덤하게 섞어주며, num_workers=0으로 설정하여 데이터 로딩에 사용되는 프로세스 개수를 지정합니다.
# train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

# # val_dataset: 검증 데이터셋 객체 생성
# val_dataset = CustomDataset(val_input, val_target)

# # val_loader: 검증 데이터셋을 DataLoader로 묶어줍니다.
# # DataLoader는 배치 크기(batch_size)만큼 데이터를 묶어주는 역할을 합니다.
# # shuffle=False로 설정하여 검증 데이터는 섞지 않고 순서대로 사용합니다.
# # num_workers=0으로 설정하여 데이터 로딩에 사용되는 프로세스 개수를 지정합니다.
# val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


### 모델 선언

In [ ]:
# BaseModel 클래스는 PyTorch의 nn.Module을 상속받아서 사용자 정의 모델을 정의하는 코드입니다.

class BaseModel(nn.Module):
    def __init__(self, input_size=5, hidden_size=512, output_size=CFG['PREDICT_SIZE']):
        super(BaseModel, self).__init__()

        # 모델의 기본 구조를 정의합니다.
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)  # LSTM 레이어를 정의합니다.
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),  # Fully connected 레이어를 정의합니다. 입력 크기를 hidden_size에서 hidden_size // 2로 줄입니다.
            nn.ReLU(),  # 활성화 함수로 ReLU를 사용합니다.
            nn.Dropout(),  # Dropout을 적용합니다. 모델을 학습할 때 일부 뉴런을 랜덤하게 끄는 것으로 과적합을 방지합니다.
            # 최종 출력 크기를 output_size로 설정합니다.
            # 수정
            nn.Linear(hidden_size//2, hidden_size//4),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//4, output_size)
        )
        self.actv = nn.ReLU()  # 모델의 출력에 ReLU 활성화 함수를 적용합니다.

    def forward(self, x):
        # 모델의 순전파를 정의합니다.
        # x shape: (B, TRAIN_WINDOW_SIZE, 5) (B: 배치 크기, TRAIN_WINDOW_SIZE: 학습 기간, 5: 입력 특성의 개수)

        batch_size = x.size(0)  # 배치 크기를 구합니다.

        # LSTM의 초기 hidden state와 cell state를 초기화합니다.
        hidden = self.init_hidden(batch_size, x.device)

        # LSTM 레이어에 입력을 주고 출력을 받습니다.
        lstm_out, hidden = self.lstm(x, hidden)

        # 마지막 시퀀스의 출력만 사용합니다.
        last_output = lstm_out[:, -1, :]

        # Fully connected 레이어에 마지막 출력을 넣고 활성화 함수를 적용합니다.
        output = self.actv(self.fc(last_output))

        return output.squeeze(1)  # 출력을 1차원으로 변환하여 반환합니다.

    def init_hidden(self, batch_size, device):
        # LSTM 레이어의 초기 hidden state와 cell state를 0으로 초기화합니다.
        return (torch.zeros(1, batch_size, self.hidden_size, device=device),
                torch.zeros(1, batch_size, self.hidden_size, device=device))


### 모델 학습

In [ ]:
def train(model, optimizer, train_loader, val_loader, device):
    # 모델을 지정한 디바이스로 이동합니다.
    model.to(device)

    # 손실 함수로 평균 제곱 오차(Mean Squared Error, MSE)를 사용합니다.
    criterion = nn.MSELoss().to(device)

    # 최적의 손실값을 저장하기 위한 변수를 초기화합니다.
    best_loss = 9999999
    best_model = None

    # 지정한 에폭(Epochs)만큼 학습을 진행합니다.
    for epoch in range(1, CFG['EPOCHS'] + 1):
        # 모델을 학습 모드로 변경합니다.
        model.train()

        # 학습 손실과 평균 절대 오차(Mean Absolute Error, MAE)를 저장하기 위한 리스트를 초기화합니다.
        train_loss = []
        train_mae = []

        # 학습 데이터 로더에서 데이터를 순회합니다.
        for X, Y in tqdm(iter(train_loader)):
            X = X.float().to(device)
            Y = Y.float().to(device)

            # 기울기를 초기화합니다.
            optimizer.zero_grad()

            # 모델에 입력 데이터를 주고 예측을 수행합니다.
            output = model(X)

            # 손실 값을 계산합니다.
            loss = criterion(output, Y)

            # 역전파를 수행하여 기울기를 계산하고 가중치를 업데이트합니다.
            loss.backward()
            optimizer.step()

            # 학습 손실과 평균 절대 오차를 저장합니다.
            train_loss.append(loss.item())

        # 검증 데이터를 사용하여 모델의 성능을 평가합니다.
        val_loss = validation(model, val_loader, criterion, device)

        # 에폭별로 학습 손실과 검증 손실을 출력합니다.
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

        # 현재까지의 최적 손실보다 더 좋은 모델이라면 모델과 손실 값을 업데이트합니다.
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')

    # 학습이 끝난 후, 가장 좋은 모델을 반환합니다.
    return best_model


In [ ]:
def validation(model, val_loader, criterion, device):
    # 모델을 평가 모드로 변경합니다.
    model.eval()

    # 검증 손실을 저장하기 위한 리스트를 초기화합니다.
    val_loss = []

    # 평가 과정에서는 기울기 계산과 파라미터 업데이트가 필요 없으므로 torch.no_grad()를 사용하여 계산을 비활성화합니다.
    with torch.no_grad():
        # 검증 데이터 로더에서 데이터를 순회합니다.
        for X, Y in tqdm(iter(val_loader)):
            # X = X.to(device)
            # Y = Y.to(device)
            X = X.float().to(device)
            Y = Y.float().to(device)
            # 모델에 입력 데이터를 주고 예측을 수행합니다.
            output = model(X)

            # 손실 값을 계산합니다.
            loss = criterion(output, Y)

            # 검증 손실을 리스트에 저장합니다.
            val_loss.append(loss.item())

    # 평균 검증 손실을 반환합니다.
    return np.mean(val_loss)


## Run !!

In [ ]:
model = BaseModel()  # BaseModel 클래스로부터 모델 객체를 생성합니다.

optimizer = torch.optim.Adam(params=model.parameters(), lr=CFG["LEARNING_RATE"])
# Adam 옵티마이저를 생성합니다. 옵티마이저의 파라미터로 모델의 파라미터들을 전달합니다.
# 학습률은 CFG 딕셔너리에 저장된 'LEARNING_RATE' 값으로 설정합니다.

infer_model = train(model, optimizer, train_loader, val_loader, device)
# train 함수를 호출하여 모델을 학습시킵니다.
# 입력으로 모델, 옵티마이저, 학습용 데이터 로더, 검증용 데이터 로더, 그리고 디바이스를 전달합니다.
# 학습이 끝난 후 최적의 모델이 반환되어 infer_model 변수에 저장됩니다.


## 모델 추론

In [ ]:
test_dataset = CustomDataset(test_input, None)
# CustomDataset 클래스를 이용하여 추론용(test) 데이터셋을 생성합니다.
# test_input은 추론을 위해 사용할 일별 판매량 데이터이며,
# test_target은 추론 과정에서는 필요하지 않기 때문에 None으로 설정합니다.

test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
# DataLoader를 사용하여 추론용 데이터셋을 미니배치 단위로 나눕니다.
# 추론 과정에서는 shuffle을 False로 설정하여 데이터를 섞지 않고,
# num_workers는 데이터 로딩에 사용할 스레드 수를 지정합니다.
# batch_size는 CFG 딕셔너리에 저장된 'BATCH_SIZE' 값으로 설정합니다.


In [ ]:
def inference(model, test_loader, device):
    # 예측 결과를 저장할 빈 리스트 생성
    predictions = []

    # 추론 과정에서는 기울기 계산이 필요 없으므로 torch.no_grad()를 사용하여 연산 최적화
    with torch.no_grad():
        # test_loader에서 배치 단위로 데이터를 가져와서 예측 수행
        for X in tqdm(iter(test_loader)):
            # X를 device(GPU 또는 CPU)로 이동
            X = X.to(device)

            # 모델에 입력 데이터 X를 전달하여 예측 수행
            output = model(X)

            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환하여 저장
            output = output.cpu().numpy()

            # 예측 결과를 predictions 리스트에 추가
            predictions.extend(output)

    # 모든 배치에 대한 예측 결과를 numpy 배열로 변환하여 반환
    return np.array(predictions)


In [ ]:
# 추론 함수를 호출하여 예측 결과를 얻습니다.
pred = inference(infer_model, test_loader, device)

# inference 함수는 학습된 모델을 사용하여 추론을 수행하는 함수입니다.
# 이 함수는 입력으로 학습된 모델인 infer_model, 추론용 입력 데이터를 가지고 있는 test_loader,
# 그리고 모델이 동작하는 디바이스(device)를 인자로 받습니다.

# 1. infer_model: 학습된 모델로서, 이전에 학습한 모델인 infer_model이 함수의 첫 번째 인자로 주어집니다.
# 이 모델은 이전 단계에서 학습한 결과를 가지고 있어서 추론을 수행할 수 있습니다.

# 2. test_loader: 추론용 입력 데이터를 배치 단위로 제공하는 DataLoader로서,
# 앞서 준비한 추론용 입력 데이터를 미니배치로 나누어 모델에 전달합니다.
# 이렇게 배치 단위로 데이터를 전달하면 모델이 한 번에 여러 데이터를 처리할 수 있어서
# 추론 속도를 향상시킬 수 있습니다.

# 3. device: 모델이 동작하는 디바이스(GPU 또는 CPU)를 지정합니다.
# 만약 CUDA 기능을 지원하는 GPU가 있다면, CUDA로 설정하여 GPU를 사용하여 추론을 가속화할 수 있습니다.
# 그렇지 않은 경우 CPU를 사용하여 추론을 수행합니다.

# 위 함수는 infer_model과 test_loader를 사용하여 추론을 수행하고, 예측 결과를 numpy 배열로 반환합니다.
# 이 결과는 pred 변수에 저장됩니다. 즉, pred에는 모델을 사용하여 추론한 예측 결과가 담겨있습니다.
# 이후 이 결과를 활용하여 원하는 후속 작업을 수행할 수 있습니다.


In [ ]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

# 결과 후처리
pred = np.round(pred, 0).astype(int)

In [ ]:
pred.shape

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit.head()

In [ ]:
submit.iloc[:,1:] = pred
submit.head()

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)